In [2]:
#这些扩展包自己安装
from PIL import Image  # 用于打开图片和对图片处理
#import pytesseract  # 用于图片转文字,这个效果一般
from selenium import webdriver  # 用于打开网站
import time  # 代码运行停顿
import sys
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
#导入期望场景类
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support import expected_conditions
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException,ElementClickInterceptedException          
import datetime
import cv2
from aip import AipOcr
import numpy as np
from PIL import ImageFilter
#半自动化爬虫，基于selenium的程序。拓目的只是为了晚起，抢场速度并不快，估摸着比正常人点击要快，但是不会被封号
#代码很low请勿吐槽，基于post和cookies肯定会更快


#图像处理修改前辈们的,有的是自己补充的，主要是自己懒得做，私认为用机器学习训练一个数据集的效果或许会更好
#可以用pyinstall打包py文件为exe
#截图验证码
def Convertimg(imglocation,driver):
#验证码的xpath地址
    driver.save_screenshot(screenshotadd)
    im = Image.open(screenshotadd)
    left = driver.find_element_by_xpath(imglocation).location['x']
    top = driver.find_element_by_xpath(imglocation).location['y']
    right = driver.find_element_by_xpath(imglocation).location['x'] + driver.find_element_by_xpath(imglocation).size['width']
    bottom = driver.find_element_by_xpath(imglocation).location['y'] + driver.find_element_by_xpath(imglocation).size['height']
    im = im.crop((left, top, right, bottom))
    im.save(codeadd)
    
#椒盐噪声处理
def del_spot(image):
    #im参数默认为灰度图
    data = cv2.imread(image)
    w,h= np.shape(data)
    black_point = 0
    for y in range(1, w - 1):
        for x in range(1, h - 1):
            mid_pixel = data[y,x]  # 中央像素点像素值
            if mid_pixel < 50:  # 找出上下左右四个方向像素点像素值
                top_pixel = data[y-1,x]
                left_pixel = data[y,x - 1]
                down_pixel = data[y + 1, x]
                right_pixel = data[y ,x + 1]
                # 判断上下左右的黑色像素点总个数
                if top_pixel < 8:
                    black_point += 1
                if left_pixel < 8:
                    black_point += 1
                if down_pixel < 8:
                    black_point += 1
                if right_pixel < 8:
                    black_point += 1
                if black_point < 1:
                    data[y,x]=255
                black_point = 0
#改变图片尺寸
def resizeImage(filein,width, height):
    img=Image.open(filein)
    out = img.resize((width, height),Image.ANTIALIAS) 
    out.save('code.png')
#清除划线
def clearimage(originadd):
    img = Image.open(originadd)#读取系统的内照片
    #将黑色干扰线替换为白色
    height = img.size[1]#高度-y
    width = img.size[0]#宽度-x
    for j in range(0,height-1):#遍历所有高度的点
        for i in range(0,width-1):#遍历所有宽度的点
            data = img.getpixel((i,j))#打印该图片的所有点
            if (data[0]<=25 and data[1]<=25 and data[2]<=25):#RGBA的r,g,b均小于25
                img.putpixel((i,j),(255,255,255,255))#则这些像素点的颜色改成白色
    img = img.convert("RGB")#把图片强制转成RGB
    img.save('code.png')#保存修改像素点后的图片
    #灰度化
    Grayimg = cv2.cvtColor(cv2.imread('code.png'), cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(Grayimg, 160, 255,cv2.THRESH_BINARY)
    cv2.imwrite('code.png', thresh)
    #提取边缘
    edimg = Image.open('code.png')
    conF = edimg.filter(ImageFilter.CONTOUR)
    conF.save("code.png")
    resizeImage('code.png',460,175)


class book:
    
    url='https://cas.sysu.edu.cn/cas/login?service=http%3A%2F%2Fgym.sysu.edu.cn%2Flogin%2Fpre.html'
    def __init__(self):
        #date\time_list\username\passwords can be set as parameters in the bracket of __init__      
        self.driver=webdriver.Chrome('/usr/local/bin/chromedriver')
        
    def yanzhengma(self,imglocation):
        #这个是我的百度识图,可以直接使用，但是建议用自己的，这个最多三个人同时用。。。
        config = {
            'appId': '23911419',
            'apiKey': 'xruZ4wmxQWIIpGpB3O3xZ7Uj',
            'secretKey': 'Yq3l5z0IS5WeU9BZG59Cln4QX0Ha0IEd'
        }
        #验证码截图
        self.driver.maximize_window()
        time.sleep(0.5)
        self.driver.save_screenshot('screen.png')
        im = Image.open('screen.png')
        left = self.driver.find_element_by_xpath(imglocation).location['x']
        top = self.driver.find_element_by_xpath(imglocation).location['y']
        right = self.driver.find_element_by_xpath(imglocation).location['x'] + self.driver.find_element_by_xpath(imglocation).size['width']
        bottom = self.driver.find_element_by_xpath(imglocation).location['y'] + self.driver.find_element_by_xpath(imglocation).size['height']
        im = im.crop((left, top, right, bottom))
        im.save('code.png')
        #图像处理
        clearimage('code.png')
        #baidu识图
        client = AipOcr(**config)
        image = open('code.png','rb').read()
        result = client.basicGeneral(image)
        if result['words_result']==[]:
            return 'asdf'
        else:
            keys=result['words_result'][0]['words']
            keys=keys.replace(' ','')
        return keys
        
    def sign_in(self,username,password):
        self.driver.get(self.url)
        t=0
        while t==0:
            #找到输入框并输入账号
            elem_id=self.driver.find_element_by_name("username")
            elem_id.clear()
            elem_id.send_keys(username)
            #同理
            elem_password=self.driver.find_element_by_name("password")
            elem_password.clear()
            elem_password.send_keys(password)
            #验证码xpath
            imglocation ='//*[@id="captchaImg"]'  # 验证码的xpath地址
            elem_yanzhengma = self.driver.find_element_by_xpath(imglocation)
            #输入验证码
            key=self.yanzhengma(imglocation)
            print(key)
            time.sleep(1.5)
            elem_yanzhenginput=self.driver.find_element_by_xpath('//*[@id="captcha"]')
            elem_yanzhenginput.clear()
            elem_yanzhenginput.send_keys(key)
            self.driver.find_element_by_xpath('//*[@id="fm1"]/section[2]/input[4]').click()
            try:
                if self.driver.find_element_by_xpath('//*[@id="fm1"]/div[1]/span').text=='验证码不正确':
                    t=0
                else:
                    t=1
            except NoSuchElementException as e:
                return
    def search(self,date,time_list):
        #date格式xxxx-xx-xx
        #time_list 为时间段的列表，格式为‘08:00-09:00'类似的
        while True:                
            try:
                time.sleep(1)
                self.driver.find_element_by_xpath('/html/body/div[1]/section/div[3]/div/a[1]').click()
                time.sleep(1)
                if self.driver.find_element_by_xpath('/html/body/div[1]/section/div[5]/div[1]/div[1]/div[1]/div[1]/div/ul/li[2]/div[3]').text==date:
                    break
                else:
                    self.driver.refresh()
                    time.sleep(1.2)
                    try:
                        if self.driver.find_element_by_xpath('//*[@id="info"]/p').text=='请到06:00-22:00再来预订！':
                            self.driver.find_element_by_xpath('//*[@id="info"]/div[2]/button[1]').click()
                    except NoSuchElementException as e:
                        print('没有这个时间提示弹窗了')
                    
            except ElementClickInterceptedException as e:
                self.driver.refresh()
                time.sleep(2)
                try:
                    if self.driver.find_element_by_xpath('//*[@id="info"]/p').text=='请到06:00-22:00再来预订！':
                        self.driver.find_element_by_xpath('//*[@id="info"]/div[2]/button[1]').click()
                except NoSuchElementException as e:
                    print('没有这个时间提示弹窗了')
        #通过观察路径的标签来判断
        self.driver.find_element_by_xpath('/html/body/div[1]/section/div[5]/div[1]/div[1]/div[1]/div[1]/div/ul/li[2]/div[1]').click()
        kong1=self.driver.find_elements_by_css_selector('[style="float: left;"]'+'[data-timer="'+time_list[0]+'"]')
        kong2=self.driver.find_elements_by_css_selector('[style="float: left;"]'+'[data-timer="'+time_list[1]+'"]')
        kong=[kong1,kong2]
        return kong
    
    def reserve(self,date,time_list):
        #先进入订场界面
        self.driver.get('http://gym.sysu.edu.cn/product/show.html?id=161')
        time.sleep(2)
        try:
            if self.driver.find_element_by_xpath('//*[@id="info"]/p').text=='请到06:00-22:00再来预订！':
                self.driver.find_element_by_xpath('//*[@id="info"]/div[2]/button[1]').click()
        except NoSuchElementException as e:
            print('没有这个时间提示弹窗了')
        
        while True:
            #返回一个列表，里面有两个列表
            label=0
            kong=self.search(date,time_list)
            for t in kong:#kong有两个时间段列表
                if t!=[]:
                    #优先后面的场
                    label=1
                    #移到要点击的位置去点击,-1是指最后一个场
                    webdriver.ActionChains(self.driver).move_to_element(t[0] ).click(t[0] ).perform()
            if label==0:
                print('一个场都没抢到，太难了')
                break
                return -1
            try:
                #点击预定
                #以下的timesleep是服务卡顿时候的情况避免重复刷新
                self.driver.find_element_by_xpath('/html/body/div[1]/section/div[5]/div[1]/div[2]/div[2]/div[3]/div/button').click()
                time.sleep(2.2)
                #点击给钱
                self.driver.find_element_by_xpath('//*[@id="reserve"]').click()
                time.sleep(3)
                #确定
                self.driver.find_element_by_xpath('/html/body/div[5]/div[2]/button[1]').click()
                time.sleep(2.5)
                self.driver.find_element_by_xpath('/html/body/div/div[2]/div[1]/div[4]/dl/dd[2]/div/ul/li').click()
                self.driver.find_element_by_xpath('/html/body/div/div[2]/div[1]/div[4]/div/button[2]').click()
                time.sleep(3)
            except NoSuchElementException:
                self.driver.refresh()
                time.sleep(1)
            except ElementNotInteractableException:
                self.driver.refresh()
                print('没的按')
                time.sleep(2)
            except ElementClickInterceptedException:
                self.driver.refresh()
                print('没的按')
                time.sleep(1.5)
            #预定结果
            try:
                if self.driver.find_element_by_xpath('//*[@id="success"]/p').text=='支付成功':
                    self.driver.find_element_by_xpath('//*[@id="success"]/div[2]/button').click()
                    break
            except NoSuchElementException as e:
                #不能refresh
                self.driver.get('http://gym.sysu.edu.cn/product/show.html?id=161')
                time.sleep(1)
                continue
        return 0

In [2]:
#该脚本用于定两天后的场
#输入账号和密码
print('metion!!!请在英文输入法下输入你的信息，否则订场会出现问题。\n')
print('输入你的netid：\n')
username=input()
print('输入你的密码：\n')
password=input()
#订场的时间
print('场地当天date格式，只可以是两天后（请在英文输入法下输入）例如：2010-09-01')
print('date:\n')
date=input()
print('订场时间：\n')
time_list=[]
for i in range(0,2):
    print('格式为，例如：19:01-20:00')
    time_list.append(input())
#等待订场
local_time=time.localtime()
now=time.strftime('%H:%M', local_time)
start=datetime.datetime.strptime(now,'%H:%M')
#以下的时间自己修改
if now>='08:00':
    mid=datetime.datetime.strptime('23:59','%H:%M')
    wait=mid-start
    time.sleep(wait.seconds+120)
print('过了0点')
    
now=time.strftime('%H:%M', time.localtime())
start=datetime.datetime.strptime(now,'%H:%M')  
end= datetime.datetime.strptime('05:58', '%H:%M')
t=end-start
time.sleep(t.seconds)
temp=book()
temp.sign_in(username,password)
#这里是修改订场的开始时间，以前有时后是晚上3点，有时是早上六点，有时是晚上12点就可以定了，不知道以后怎么样，请自己修改
while time.strftime('%H:%M', time.localtime())<='05:59':
    time.sleep(10)
print('开始：')
time.sleep(5)
#开始预定
temp.reserve(date,time_list)

metion!!!请在英文输入法下输入你的信息，否则订场会出现问题。

输入你的netid：



KeyboardInterrupt: Interrupted by user